<a href="https://colab.research.google.com/github/tanoManzo/mimic_trajectories/blob/dev/MIMIC_TRAJECTORY_embeddings_03_collect_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Collection, Embeddings and Attitude

In [7]:
! pip install transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.5 MB/s eta 0:00:00


In [26]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from transformers import AutoModelForSequenceClassification
from sklearn.metrics.pairwise import cosine_similarity

NameError: ignored

load nursing notes subject-caregiver-chartdate

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
notes_path = "/content/drive/MyDrive/NIH/Data/"
notes_name = "NOTES_NURSING_TRAJECTORIES_01.csv"
df_notes = pd.read_csv(notes_path+notes_name)
df_notes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 531675 entries, 0 to 531674
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   ROW_ID      531675 non-null  int64 
 1   SUBJECT_ID  531675 non-null  int64 
 2   CGID        531675 non-null  int64 
 3   CHARTDATE   531675 non-null  object
 4   TEXT        531675 non-null  object
dtypes: int64(3), object(2)
memory usage: 20.3+ MB


In [13]:
ward_name = "CALLOUT.csv"
df_ward = pd.read_csv(notes_path+ward_name)
df_ward.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34499 entries, 0 to 34498
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ROW_ID                  34499 non-null  int64  
 1   SUBJECT_ID              34499 non-null  int64  
 2   HADM_ID                 34499 non-null  int64  
 3   SUBMIT_WARDID           34498 non-null  float64
 4   SUBMIT_CAREUNIT         5122 non-null   object 
 5   CURR_WARDID             34498 non-null  float64
 6   CURR_CAREUNIT           34498 non-null  object 
 7   CALLOUT_WARDID          34499 non-null  int64  
 8   CALLOUT_SERVICE         34499 non-null  object 
 9   REQUEST_TELE            34499 non-null  int64  
 10  REQUEST_RESP            34499 non-null  int64  
 11  REQUEST_CDIFF           34499 non-null  int64  
 12  REQUEST_MRSA            34499 non-null  int64  
 13  REQUEST_VRE             34499 non-null  int64  
 14  CALLOUT_STATUS          34499 non-null

In [15]:
ward_name = "ICUSTAYS.csv"
df_ward = pd.read_csv(notes_path+ward_name)
df_ward.info()
df_ward['FIRST_CAREUNIT'].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61532 entries, 0 to 61531
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ROW_ID          61532 non-null  int64  
 1   SUBJECT_ID      61532 non-null  int64  
 2   HADM_ID         61532 non-null  int64  
 3   ICUSTAY_ID      61532 non-null  int64  
 4   DBSOURCE        61532 non-null  object 
 5   FIRST_CAREUNIT  61532 non-null  object 
 6   LAST_CAREUNIT   61532 non-null  object 
 7   FIRST_WARDID    61532 non-null  int64  
 8   LAST_WARDID     61532 non-null  int64  
 9   INTIME          61532 non-null  object 
 10  OUTTIME         61522 non-null  object 
 11  LOS             61522 non-null  float64
dtypes: float64(1), int64(6), object(5)
memory usage: 5.6+ MB


array(['MICU', 'CCU', 'NICU', 'TSICU', 'SICU', 'CSRU'], dtype=object)

## **Data Preprocessing**

extract nicu notes

In [19]:
def nicu_condition(x):
  words = ['nicu', 'neonatal']
  return any(word in x.lower() for word in words)

# get list of subject in nicu    
# nicu_subject_ids = df_notes[df_notes['TEXT'].apply(lambda x: nicu_condition(x))]['SUBJECT_ID'].unique()

# get nicu notes
df_notes_nicu = df_notes.merge(df_ward, on='SUBJECT_ID')
#df_notes_nicu = df_notes[df_notes['SUBJECT_ID'].isin(nicu_subject_ids)]
df_notes_nicu = df_notes_nicu[df_notes_nicu['FIRST_CAREUNIT'] == 'TSICU']
df_notes_nicu

,ROW_ID_x,SUBJECT_ID,CGID,CHARTDATE,TEXT,ROW_ID_y,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
3299,1373553,8406,17765,2149-04-28,SOCIAL WORK NOTE:\n\nNew trauma pt on T-SICU f...,10680,114444,239553,carevue,TSICU,TSICU,33,33,2149-04-25 22:04:41,2149-05-22 14:04:55,26.6668
3301,1373550,8406,16140,2149-04-27,SICU NPN\nROS:\nNeuro: Neuro exam unchanged th...,10680,114444,239553,carevue,TSICU,TSICU,33,33,2149-04-25 22:04:41,2149-05-22 14:04:55,26.6668
3303,1373547,8406,16140,2149-04-26,"SICU NPN\nPt hemodynamicallly unstable today, ...",10680,114444,239553,carevue,TSICU,TSICU,33,33,2149-04-25 22:04:41,2149-05-22 14:04:55,26.6668
3305,1373567,8406,15659,2149-05-01,T/SICU NSG PROGRESS NOTE.\n0700>>[**2076**]\n\...,10680,114444,239553,carevue,TSICU,TSICU,33,33,2149-04-25 22:04:41,2149-05-22 14:04:55,26.6668
3307,1373633,8406,21570,2149-07-02,Nursing Progress Note.\n\nCV: The pt has been ...,10680,114444,239553,carevue,TSICU,TSICU,33,33,2149-04-25 22:04:41,2149-05-22 14:04:55,26.6668
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
908103,1672405,32374,18448,2110-05-22,Admission Note:\n\nPt is a 76 yo male admitted...,40789,141164,277958,carevue,TSICU,TSICU,14,14,2110-05-22 17:26:38,2110-05-22 23:51:15,0.2671
908196,1587513,24935,14441,2192-11-17,Phone contact with pt's daughter\n[**Name (NI)...,31568,141756,281113,carevue,TSICU,TSICU,33,33,2192-11-16 21:47:30,2192-11-17 23:42:36,1.0799
908271,1410250,11231,18078,2140-08-28,TRAUMA ICU NURSING PROGRESS NOTE\n\nPT ARRIVED...,14286,183363,275862,carevue,TSICU,TSICU,57,57,2140-08-27 19:50:32,2140-08-28 02:19:38,0.2702
908445,1390928,9769,19611,2150-12-20,admission note\ndata: admitted to the trauma s...,12427,169187,291029,carevue,TSICU,TSICU,14,14,2150-12-20 19:13:43,2150-12-20 21:39:43,0.1014


## Data exploration 

check number of notes per patient/subject

In [ ]:
import seaborn as sns
sns.set()


gb_notes = df_notes_nicu.groupby(['CGID','SUBJECT_ID'])
num_notes = gb_notes['TEXT'].count()

print(f'number of notes: min={num_notes.min()}, max={num_notes.max()}, mean={num_notes.mean()},  median={num_notes.median()}')
sns.boxplot(x=num_notes)

check number of caregiver/subject that have between 4 and 15 notes

In [ ]:
sns.displot(num_notes[((num_notes>2) & (num_notes<15))])

## Embedding and Attitude

### Input data

In [20]:
cg_subjects_dict = {}
caregivers_ids = df_notes_nicu['CGID'].unique()

for cg_id in caregivers_ids:
  cg_subjects_dict[cg_id] = df_notes_nicu[df_notes_nicu['CGID']==cg_id]['SUBJECT_ID'].unique().tolist()

In [21]:
# loop through len(ids)
def create_sentences(cgid=14098, subid=13155):
  df_notes_nicu_selected = df_notes_nicu[(df_notes_nicu['CGID'] == cgid) & (df_notes_nicu['SUBJECT_ID'] == subid)].sort_values(by=['CHARTDATE'])
  sentences = df_notes_nicu_selected['TEXT'].tolist()
  return df_notes_nicu_selected, sentences

df_notes_nicu_selected, sentences = create_sentences()

In [22]:
df_notes_nicu_selected

,ROW_ID_x,SUBJECT_ID,CGID,CHARTDATE,TEXT,ROW_ID_y,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS


### Embeddings 

embedding model

In [23]:
model_ckpt = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

RuntimeError: ignored

get embeddings

In [ ]:
def create_dist_data(list_of_sentences):  
  encoded_input = tokenizer(list_of_sentences, padding=True, truncation=True, return_tensors="pt")
  with torch.no_grad():
    model_output = model(**encoded_input)
  token_embeddings = model_output.last_hidden_state  

  def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

  # Perform pooling
  sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

  # Normalize embeddings
  sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

  sentence_embeddings_np = sentence_embeddings.detach().numpy()

  scores = np.zeros((sentence_embeddings_np.shape[0],sentence_embeddings_np.shape[0]))
  dist = []
  for idx in range(sentence_embeddings_np.shape[0]):
    scores[idx,:] = cosine_similarity([sentence_embeddings_np[idx]],sentence_embeddings_np)[0]            
    if idx < len(sentence_embeddings_np)-1:
      dist.append(scores[idx,idx+1])
  return dist

dist = create_dist_data(sentences)

In [ ]:
dist

### Attitude 

attitude models

In [ ]:
# model name from huggingface.co/model name_id:model_name
models_name = {
  'roberta':'tanoManzo/roberta-attitude',
#  'distilbert':'tanoManzo/distilbert-attitude',
#  'minilm':'tanoManzo/minilm-attitude',
#  'bloom':'tanoManzo/bloom-attitude'
}

# load tokenizer and model 
tokenizers = [AutoTokenizer.from_pretrained(model_name) for model_name in models_name.values()]
models= [AutoModelForSequenceClassification.from_pretrained(model_name) for model_name in models_name.values()]

In [ ]:
def create_sentiment_labels(df_notes_nicu_selected, list_of_sentences):
  # select dataframe split
  df_to_infer = df_notes_nicu_selected

  # parameters to set 
  total_notes = len(df_to_infer)
  batch_size = 1
  iterations = int(len(df_to_infer)/batch_size)

  # start label creation
  l_scores = {k:[] for k in models_name.keys()} # dict Model: labels 
  l_labels = {k:[] for k in models_name.keys()} # dict Model: scoress
  #print(iterations)
  for iter in range(iterations):
    

    idx_start = batch_size*iter     # first window sentence idx in the dataframe
    idx_end = batch_size*(iter+1)   # last window sentence idx in the dataframe
    
    # get row sentences for batch
    list_of_sentences = list(df_to_infer['TEXT'].iloc[idx_start:idx_end].values)
    #print(len(models_name))
    for (index, (key, value)) in enumerate(models_name.items()):
      
      # tokenized input for the model 
      tokenizer = tokenizers[index]
      batch = tokenizer(list_of_sentences, # list of sentence 
                      padding=True, # add if short 
                      truncation=True, # remove if long 
                      max_length=512, # sent length
                      return_tensors="pt" # to return pytorch tensor (NO for TF)
                      )
      #print(batch)        
      model = models[index]
      with torch.no_grad():
        outputs = model(**batch)
        predictions = F.softmax(outputs.logits, dim=1)
        label_ids = torch.argmax(predictions, dim=1)

        labels = [model.config.id2label[label_id] for label_id in label_ids.tolist()]
        scores = [round(float(predictions[idx][label_id].item()),4) for idx, label_id in enumerate(label_ids.tolist())]
        l_labels[key].extend(labels)
        l_scores[key].extend(scores)
      #print(l_labels)
  return l_labels

l_labels = create_sentiment_labels(df_notes_nicu_selected,sentences)

In [ ]:
l_labels

In [ ]:
df_notes_nicu_selected

# **Add more data**

Group data and count groupings

In [ ]:
#grouped_df = df_notes_nicu.groupby(['CGID','SUBJECT_ID']).size().reset_index(name='count')
grouped_df = df_notes_nicu.groupby(['CGID']).size().reset_index(name='count')
grouped_df.tail(3)
#len(grouped_df['CGID'].unique())

Select samples within count ranges

In [ ]:
percentiles_cgid_subid = [(0.8, 0.85),(0.85, 0.9),(0.9, 0.95),(0.95, 0.975),(0.975, 0.98)]
percentiles_cgid_only = [(0.5, 0.6),(0.6, 0.7),(0.7, 0.75),(0.75, 0.8),(0.8, 0.85)]

filtered_rows = []
for min_pct, max_pct in percentiles_cgid_only:
  min = grouped_df['count'].quantile(min_pct)
  max = grouped_df['count'].quantile(max_pct)

  range_df = grouped_df[(grouped_df['count'] >= min) & (grouped_df['count'] <= max)]

  row = range_df.sample(n=12)
  filtered_rows.append(row)
  
result = pd.concat(filtered_rows)

print(result)
print(list(result['CGID']))
cgids = list(result['CGID'])

In [ ]:
# write data to rows
df_to_plot = pd.DataFrame(columns = ['CGID','SUBJECT_ID','emb_distance','attitudes'])
#cgids = [14098]
print(cgids)
for cgid in cgids:
  for subject_id in cg_subjects_dict[cgid]:
    df_notes_nicu_selected, sentence = create_sentences(cgid=cgid, subid=subject_id)
    dist = create_dist_data(sentence)
    l_labels = create_sentiment_labels(df_notes_nicu_selected,sentence)
    print([cgid, subject_id, dist, list(*l_labels.values())])
    df_to_plot.loc[len(df_to_plot)] = [cgid, subject_id, dist, list(*l_labels.values())]

In [ ]:
# path and name of the csv file 
PATH_TO_SAVE = "/content/drive/MyDrive/NIH/Data/"
name_file_to_save = 'NOTES_NURSING_TRAJECTORIES_sixtyCGs.csv'
#df_to_plot.to_csv(PATH_TO_SAVE+name_file_to_save,index=False)

In [ ]:
df_to_plot

# END 


# **Add labels to graph**

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.ticker import MaxNLocator

In [ ]:
sns.scatterplot(dist)
model_labels = l_labels['roberta']
labels_dict = {'Overall Neutral Note': '0', 'Overall Negative Note': 'N', 'Overall Positive Note': 'P'}
graph_labels = [labels_dict[label] for label in model_labels]

# Add sentiment next to points
for index, row in enumerate(dist):

  x_position = index - 0.15
  x2_position = index + 0.075
  y_position = dist[index] - 0.0075
  plt.text(x_position, y_position, graph_labels[index], fontsize=10, color='red')
  plt.text(x2_position, y_position, graph_labels[index+1], fontsize=10, color='red')

# Title and x/y axes
plt.title("Sentence Comparison of Cosine Similarity and Sentiment")
plt.xlabel("Sentence Comparison")
plt.ylabel("Cosine Similarity")

# X-axis tick marks
ax = plt.gca()
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
new_ticks = [str(i) for i in range(len(graph_labels))]
ax.set_xticklabels(new_ticks)

# Legend
legend_handles = [Rectangle((0, 0), 0, 0, alpha=0, label=f"{short_label}: {original_label}") 
                  for original_label, short_label in labels_dict.items()]
#legend_handles = [Text(0, 0, label=f"{short_label}: {original_label}") for original_label, short_label in labels_dict.items()]

# legend to right of graph
#plt.legend(handles=legend_handles, loc='center left', bbox_to_anchor=(1, 0.5), title='Sentiment')

# legend under graph
plt.legend(handles=legend_handles, loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=len(labels_dict), title='Sentence Sentiment')

plt.show()

In [ ]:
sns.scatterplot(dist)
model_labels = l_labels['roberta']
labels_dict = {'Overall Neutral Note': '0', 'Overall Negative Note': 'N', 'Overall Positive Note': 'P'}
graph_labels = [labels_dict[label] for label in model_labels]

# Add sentiment next to points
plt.text(-0.15, dist[0]-.0075, graph_labels[0], fontsize=10, color='red')
for index, row in enumerate(dist):
  if index == 0:
    continue
  x_position = index - 0.5
  x2_position = index + 0.075
  y_position = (dist[index] + dist[index-1] - 0.0075)/2
  plt.text(x_position, y_position, graph_labels[index], fontsize=10, color='red')
plt.text(len(dist)-1+0.075, dist[len(dist)-1]-0.0075, graph_labels[len(dist)], fontsize=10, color='red')

# Title and x/y axes
plt.title("Sentence Comparison of Cosine Similarity and Sentiment")
plt.xlabel("Sentence Comparison")
plt.ylabel("Cosine Similarity")

# X-axis tick marks
ax = plt.gca()
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
new_ticks = [str(i) for i in range(len(graph_labels))]
ax.set_xticklabels(new_ticks)

# Legend
legend_handles = [Rectangle((0, 0), 0, 0, alpha=0, label=f"{short_label}: {original_label}") 
                  for original_label, short_label in labels_dict.items()]
#legend_handles = [Text(0, 0, label=f"{short_label}: {original_label}") for original_label, short_label in labels_dict.items()]

# legend to right of graph
#plt.legend(handles=legend_handles, loc='center left', bbox_to_anchor=(1, 0.5), title='Sentiment')

# legend under graph
plt.legend(handles=legend_handles, loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=len(labels_dict), title='Sentence Sentiment')

plt.show()

## Clustering 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

In [ ]:
clustering_model = KMeans(n_clusters=3)
clustering_model.fit(sentence_embeddings_np)
cluster_assignment = clustering_model.labels_
print(cluster_assignment)

In [ ]:
clustered_sentences = {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in clustered_sentences:
        clustered_sentences[cluster_id] = []

    clustered_sentences[cluster_id].append(sentences[sentence_id])
clustered_sentences

In [ ]:
from scipy.spatial.distance import cdist

distortions = []
inertias = []
mapping1 = {}
mapping2 = {}
K = range(1, 6)
X = sentence_embeddings_np 
for k in K:
    # Building and fitting the model
    kmeanModel = KMeans(n_clusters=k).fit(X)
    kmeanModel.fit(X)
  
    distortions.append(sum(np.min(cdist(X, kmeanModel.cluster_centers_,
                                        'cosine'), axis=1)) / X.shape[0])
    inertias.append(kmeanModel.inertia_)
  
    mapping1[k] = sum(np.min(cdist(X, kmeanModel.cluster_centers_,
                                   'euclidean'), axis=1)) / X.shape[0]
    mapping2[k] = kmeanModel.inertia_

In [ ]:
import matplotlib.pyplot as plt
plt.plot(K, distortions, 'bx-')
plt.xlabel('Values of K')
plt.ylabel('Distortion')
plt.title('The Elbow Method using Distortion')
plt.show()